In [1]:
## import libaries
import pandas as pd
import numpy as np
import cv2
import os, sys
from tqdm import tqdm
from keras.utils import to_categorical

Using TensorFlow backend.


In [2]:
def read_img(img_path):
    img = cv2.imread(img_path, cv2.IMREAD_COLOR)
    img = cv2.resize(img, (224,224))
    return img

In [3]:
train_consanant = pd.read_csv('Data/Consonant.csv')
train_vowel = pd.read_csv('Data/Vowel.csv')

In [4]:
TRAIN_PATH = 'Data/'

In [5]:
train_img = []
for img_path in tqdm(train_consanant['FileName'].values):
    train_img.append(read_img(TRAIN_PATH +"Consonant/"+ img_path))
for img_path in tqdm(train_vowel['FileName'].values):
    train_img.append(read_img(TRAIN_PATH +"Vowel/"+ img_path))    

100%|███████████████████████████████████████████████████████████████████████████████| 275/275 [00:02<00:00, 100.84it/s]


In [6]:
x_train = np.array(train_img, np.float32) / 255.

In [7]:
x_train.shape

(1430, 224, 224, 3)

In [8]:
y_train = [0]*1430

In [9]:
for x in range(1155,1430):
    y_train[x] = 1

In [10]:
y_train = np.array(y_train)
y_train = to_categorical(y_train)

In [11]:
from sklearn.cross_validation import train_test_split

In [12]:
X_train,X_test,y_train,y_test = train_test_split(x_train,y_train)

In [13]:
from keras.models import Sequential
from keras.layers import Convolution2D, MaxPooling2D,Conv2D,BatchNormalization
from keras.layers import Activation, Dropout, Flatten, Dense

In [14]:
img_width, img_height = 224,224

input_shape = (img_width, img_height, 3)

model = Sequential()

model.add(Conv2D(96, kernel_size=(11, 11), strides=(4, 4), activation='relu', input_shape=(224, 224, 3)))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(5, 5), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Conv2D(256, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(Conv2D(384, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
model.add(BatchNormalization())

model.add(Flatten())
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))
model.add(Dense(4096, activation='tanh'))
model.add(Dropout(0.5))

model.add(Dense(2, activation='softmax'))


In [15]:
model.compile(loss = 'binary_crossentropy', optimizer = 'adam', metrics = ['accuracy'])

In [16]:
nb_epoch = 10
nb_train_samples = 64

In [17]:
from keras.callbacks import ModelCheckpoint

In [18]:
filepath ="weights.{epoch:02d}-{val_acc:.2f}.hdf5"

In [19]:
modelCheckpnt =ModelCheckpoint(filepath=filepath,monitor='val_acc',verbose=1,save_best_only=True,save_weights_only=False,period=1)

In [20]:
from keras.callbacks import TensorBoard

In [21]:
Tboard = TensorBoard(log_dir='./logs', histogram_freq=0, batch_size=64, write_graph=True, write_grads=False, write_images=False, embeddings_freq=0, embeddings_layer_names=None, embeddings_metadata=None)

In [22]:
model.fit(X_train,y_train,batch_size=64,epochs=50,verbose=1,validation_data=(X_test,y_test),callbacks=[modelCheckpnt,Tboard])

Train on 1072 samples, validate on 358 samples
Epoch 1/50
1072/1072 [==============================] - 6s - loss: 2.9434 - acc: 0.6446 - val_loss: 1.0587 - val_acc: 0.8128
Epoch 2/50
1072/1072 [==============================] - 2s - loss: 0.8074 - acc: 0.7435 - val_loss: 1.3051 - val_acc: 0.3520
Epoch 3/50
1072/1072 [==============================] - 2s - loss: 0.4750 - acc: 0.7957 - val_loss: 1.2990 - val_acc: 0.2011
Epoch 4/50
1072/1072 [==============================] - 2s - loss: 0.4237 - acc: 0.8228 - val_loss: 0.7983 - val_acc: 0.2235
Epoch 5/50
1072/1072 [==============================] - 2s - loss: 0.3695 - acc: 0.8582 - val_loss: 1.2816 - val_acc: 0.1872
Epoch 6/50
1072/1072 [==============================] - 2s - loss: 0.3233 - acc: 0.8713 - val_loss: 2.0320 - val_acc: 0.1872
Epoch 7/50
1072/1072 [==============================] - 2s - loss: 0.2688 - acc: 0.8834 - val_loss: 0.8650 - val_acc: 0.3128
Epoch 8/50
1072/1072 [==============================] - 2s - loss: 0.2923 - ac

In [23]:
from sklearn.metrics import f1_score

In [24]:
y_pred = model.predict_classes(X_test)

358/358 [==============================] - 0s     


In [25]:
y_pred = to_categorical(y_pred)

In [26]:
print(f1_score(y_test,y_pred))

C:\Users\KAMALRAJ\Anaconda3\lib\site-packages\sklearn\metrics\classification.py:756: DeprecationWarning: The default `weighted` averaging is deprecated, and from version 0.18, use of precision, recall or F-score with multiclass or multilabel data or pos_label=None will result in an exception. Please set an explicit value for `average`, one of (None, 'micro', 'macro', 'weighted', 'samples'). In cross validation use, for instance, scoring="f1_weighted" instead of scoring="f1".
  sample_weight=sample_weight)


0.94326446055


In [27]:
from sklearn.metrics import classification_report

In [28]:
print(classification_report(y_test,y_pred,target_names=['Consonant','Vowel']))

             precision    recall  f1-score   support

  Consonant       0.99      0.94      0.96       291
      Vowel       0.79      0.94      0.86        67

avg / total       0.95      0.94      0.94       358

